##1. Install and Import Dependencies

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

##2. Instantiate Model

In [ ]:
!pip uninstall torchvision -y
!pip install torchvision==0.21.0+cu118 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip uninstall transformers -y
!pip install transformers

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#

##3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. GREAT', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
int(torch.argmax(result.logits))+1

##4.Collect Reviews

In [ ]:
# Fetch the page
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')

# Parse the HTML
soup = BeautifulSoup(r.text, 'html.parser')

# Define regex to match review comments
regex = re.compile('.*comment.*')

# Find all relevant <p> tags with class matching the regex
results = soup.find_all('p', {'class': regex})

# Extract the text from each review
reviews = [result.text.strip() for result in results if result.text.strip()]

# Print the reviews
for review in reviews:
    print(review)

##5. Load Reviews into Dataframe and Score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['review'].iloc[0]

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [ ]:
sentiment_score(df['review'].iloc[1])

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))


In [ ]:
df

In [ ]:
df['review'].iloc[3]